In [1]:
from decouple import config

In [2]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [3]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [4]:
import requests

In [5]:
api_key = ALPHA_VANTAGE_API_KEY

params = {
    "api_key": ALPHA_VANTAGE_API_KEY,
    "ticker": "AAPL",
    "function": "TIME_SERIES_INTRADAY"
}

url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval=5min&apikey={api_key}'.format(
    **params
)
r = requests.get(url)
data = r.json()

In [7]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
dataset_key

'Time Series (5min)'

In [8]:
results = data[dataset_key] # dict
# for timestamp_str in results.keys():
#     print(timestamp_str, results.get(timestamp_str))

In [9]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2024-12-10 19:55:00'

In [10]:
result_val = results.get(timestamp_str)
result_val

{'1. open': '247.6500',
 '2. high': '247.7400',
 '3. low': '247.6100',
 '4. close': '247.6900',
 '5. volume': '750'}

In [11]:
import pytz
from datetime import datetime
from decimal import Decimal

def transform_alpha_vantage_result(timestamp_str, result):
    # unix_timestamp = result.get('t') / 1000.0
    # utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))
    timestamp_format = '%Y-%m-%d %H:%M:%S' 
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(datetime.strptime(timestamp_str,timestamp_format)).astimezone(utc)
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': int(result['5. volume']),
        'volume_weighted_average': None,
        'time': timestamp,
    }

In [12]:
transform_alpha_vantage_result(timestamp_str, result_val)

{'open_price': Decimal('247.6500'),
 'close_price': Decimal('247.6900'),
 'high_price': Decimal('247.7400'),
 'low_price': Decimal('247.6100'),
 'number_of_trades': None,
 'volume': 750,
 'volume_weighted_average': None,
 'time': datetime.datetime(2024, 12, 11, 0, 55, tzinfo=<UTC>)}

In [13]:
for timestamp_str in results.keys():
    new_data = transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
    print(new_data)

{'open_price': Decimal('247.6500'), 'close_price': Decimal('247.6900'), 'high_price': Decimal('247.7400'), 'low_price': Decimal('247.6100'), 'number_of_trades': None, 'volume': 750, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 12, 11, 0, 55, tzinfo=<UTC>)}
{'open_price': Decimal('247.6600'), 'close_price': Decimal('247.6900'), 'high_price': Decimal('247.7400'), 'low_price': Decimal('247.6100'), 'number_of_trades': None, 'volume': 1364, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 12, 11, 0, 50, tzinfo=<UTC>)}
{'open_price': Decimal('247.6200'), 'close_price': Decimal('247.6750'), 'high_price': Decimal('247.7300'), 'low_price': Decimal('247.5500'), 'number_of_trades': None, 'volume': 340, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 12, 11, 0, 45, tzinfo=<UTC>)}
{'open_price': Decimal('247.5500'), 'close_price': Decimal('247.6200'), 'high_price': Decimal('247.7200'), 'low_price': Decimal('247.5500'), 'number_of_trades': None, '